Welcome!  If you are new to Google Colab/Jupyter notebooks, you might take a look at [this notebook](https://colab.research.google.com/notebooks/basic_features_overview.ipynb) first.

**I recommend you run the first code cell of this notebook immediately, to start provisioning drake on the cloud machine, then you can leave this window open as you [read the textbook](http://underactuated.csail.mit.edu/trajopt.html).**

# Notebook Setup

The following cell will:
- on Colab (only), install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.  This will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  If you navigate between notebooks using Colab's "File->Open" menu, then you can avoid provisioning a separate machine for each notebook.
- import packages used throughout the notebook.

You will need to rerun this cell if you restart the kernel, but it should be fast (even on Colab) because the machine will already have drake installed.

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

# Install drake (and underactuated).
if 'google.colab' in sys.modules and importlib.util.find_spec('underactuated') is None:
    urlretrieve(f"http://underactuated.csail.mit.edu/scripts/setup/setup_underactuated_colab.py",
                "setup_underactuated_colab.py")
    from setup_underactuated_colab import setup_underactuated
    setup_underactuated(underactuated_sha='15cfd96b0bdfd1b0c67597c24f91907776c02a6d', drake_version='0.27.0', drake_build='release')


NOTE: This notebook contains interactive widgets that will work on Binder but not on Google Colab.  As such, the standard Colab setup script is not included.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact
from pydrake.all import MathematicalProgram, Solve, eq, le, ge

from underactuated.jupyter import SetupMatplotlibBackend
SetupMatplotlibBackend()

prog = MathematicalProgram()

lp_relaxation = False
N = 20     # number of timesteps
h = 0.1    # timestep
bigM = 10 
q_start = np.array([0., -2.])
q_goal = np.array([0., 2.])

q = prog.NewContinuousVariables(2, N, "q")   # positions
v = prog.NewContinuousVariables(2, N, "v")   # velocities
u = prog.NewContinuousVariables(2, N-1, "u") # acceleration command
if lp_relaxation:
  b = prog.NewContinuousVariables(4, N-1, "b") # binaries for collisions
  prog.AddBoundingBoxConstraint(0, 1, b)
else:
  b = prog.NewBinaryVariables(4, N-1, "b")     # binaries for collisions

# Initial value constraint:
start = prog.AddLinearConstraint(eq(q[:,0], q_start))
prog.AddLinearConstraint(eq(v[:,0], [0., 0.]))

# Final value constraint:
goal = prog.AddLinearConstraint(eq(q[:,N-1], q_goal))
prog.AddLinearConstraint(eq(v[:,N-1], [0., 0.]))

for n in range(0, N-1):
  # Dynamics constraints:
  prog.AddLinearConstraint(eq(q[:,n+1], q[:,n] + h*v[:,n]))
  prog.AddLinearConstraint(eq(v[:,n+1], v[:,n] + h*u[:,n]))
  
  # Stage cost: sum u^2
  prog.AddQuadraticCost(u[:,n].dot(u[:,n]))
  
  # Collision avoidance constraints:
  prog.AddLinearConstraint(q[0,n+1] >= 1 - (1-b[0,n])*bigM)
  prog.AddLinearConstraint(q[0,n+1] <= -1 + (1-b[1,n])*bigM)
  prog.AddLinearConstraint(q[1,n+1] >= 1 - (1-b[2,n])*bigM)
  prog.AddLinearConstraint(q[1,n+1] <= -1 + (1-b[3,n])*bigM)
  prog.AddLinearConstraint(np.sum(b[:,n]) >= 1)
 
# TODO(russt): This currently fails with only open-source solvers.  See drake #13299
result = Solve(prog)
  
qt = result.GetSolution(q)
bt = result.GetSolution(b)

fig, ax = plt.subplots(2, 1)
line, = ax[0].plot(qt[0,:], qt[1,:], 'b.-')
start_pt, = ax[0].plot(q_start[0], q_start[1], 'b*', markersize=15)
goal_pt, = ax[0].plot(q_goal[0], q_goal[1], 'g*', markersize=15)
ax[0].fill([1, 1, -1, -1], [-1, 1, 1, -1], 'r')
ax[0].axis('equal')
ax[0].set_ylim(-4, 4)
blines = ax[1].plot(bt.T)
ax[1].set_ylim(-.1, 1.1)
ax[1].set_xticks(range(0,N,2))
ax[1].legend(blines, ("right", "left", "top", "bottom"))

def resolve(start_x, start_y, goal_x, goal_y):
  start.evaluator().UpdateLowerBound([start_x, start_y])
  start.evaluator().UpdateUpperBound([start_x, start_y])
  goal.evaluator().UpdateLowerBound([goal_x, goal_y])
  goal.evaluator().UpdateUpperBound([goal_x, goal_y])

  result = Solve(prog)
  print('Feasible: ' + str(result.is_success()))

  qt = result.GetSolution(q)
  bt = result.GetSolution(b)
  
  line.set_xdata(qt[0,:])
  line.set_ydata(qt[1,:])
  start_pt.set_xdata(start_x)
  start_pt.set_ydata(start_y)
  goal_pt.set_xdata(goal_x)
  goal_pt.set_ydata(goal_y)
  for i in range(4):
    blines[i].set_ydata(bt[i,:])
  fig.canvas.draw()
  
interact(resolve, start_x=(-4,4,0.1), start_y=(-4,-1,0.1), goal_x=(-4,4,0.1), goal_y=(0,4,0.1));